In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
sqlite_uri = 'sqlite:///../ui/database/data.db'
engine = create_engine(sqlite_uri)
session = sessionmaker(autocommit=False, autoflush=False, bind=engine)
db = session()

In [3]:
db

## Ingest Data Profile Ideal Tanaman

In [4]:
import pandas as pd

df_tanaman = pd.read_csv("../data/data_profile_tanaman_w_interpolasi.csv", sep=';')
df_tanaman.head()

,jenis_tanaman,suhu,suhu_interpolasi,curah_hujan,curah_hujan_interpolasi,kelembapan,kelembapan_interpolasi,jenis_tanah,tekstur_tanah,ph,ph_interpolasi,kemiringan,kemiringan_interpolasi,topografi,topografi_interpolasi
0,Bawang Merah,10-30,"0,25,32,57",350-800,"0,300,2500,2800",80-90,"0,80,90,170","Alluvial, Glei Humus, Latosol","Liat berpasir, liat, liat berdebu, Lempung ber...",6-8,"0,5.6,6.5,12.1",5.5-16,"0,30,60",700-1000,"0,700,1000,1700"
1,Cabai Merah,18-27,"0,18,26,44",600-1400,"0,600,1200,1800",18-30,"0,18,30,48","Alluvial, Glei Humus, Latosol","Liat berpasir, liat, liat berdebu, Lempung ber...",6-8,"0,5.5,6.8,12.3",5.5-16,"0,25,50",900-1800,"0,900,1800,2700"
2,Cabai Rawit,18-27,"0,18,27,44",600-1400,"0,600,1400,2000",18-30,"0,18,30,48","Alluvial, Glei Humus, Latosol","Liat berpasir, liat, liat berdebu, Lempung ber...",6-8,"0,6.0,8.0,14.0",5.5-16,"0,8,16,25",900-1800,"0,900,1800,2700"
3,Tomat,18-30,"0,18,26,44",400-800,"0,400,700,1100",24-90,"0,35,80,115","Andosol, Latosol, Regosol","Liat berpasir, liat, liat berdebu, Lempung ber...",6-8,"0,5.5,7.0,12.5",5.5-8,"0,45,90",350-750,"0,350,750,1100"
4,Wortel,16-20,"0,16,20,45",65-80,"0,65,80,120",40-90,"0,40,90,120","Alluvial, Andosol, Latosol, Regosol","Lempung berliat, lempung liat berpasir, lempun...",5.6-8,"0,5.6,8,14",5.5-8,"0,8,16,25",1000-1200,"0,1000,1200,1600"


In [5]:
df_tanaman.columns

Index(['jenis_tanaman', 'suhu', 'suhu_interpolasi', 'curah_hujan',
       'curah_hujan_interpolasi', 'kelembapan', 'kelembapan_interpolasi',
       'jenis_tanah', 'tekstur_tanah', 'ph', 'ph_interpolasi', 'kemiringan',
       'kemiringan_interpolasi', 'topografi', 'topografi_interpolasi'],
      dtype='object')

In [6]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DateTime

Base = declarative_base()


class Tanaman(Base):
    __tablename__ = "tanaman"

    id = Column(Integer, index=True, primary_key=True, autoincrement=True)
    jenis_tanaman = Column(String)
    suhu = Column(String)
    suhu_interpolasi = Column(String)
    curah_hujan = Column(String)
    curah_hujan_interpolasi = Column(String)
    kelembapan = Column(String)
    kelembapan_interpolasi = Column(String)
    jenis_tanah = Column(String)
    tekstur_tanah = Column(String)
    ph = Column(String)
    ph_interpolasi = Column(String)
    kemiringan = Column(String)
    kemiringan_interpolasi = Column(String)
    topografi = Column(String)
    topografi_interpolasi = Column(String)


/var/folders/4w/rtqyy6vs1lj9kwm2vwrbblt40000gn/T/ipykernel_61031/1733893927.py:4: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [30]:
Base.metadata.create_all(engine)

In [31]:
tanamans = []

for id, item in df_tanaman.iterrows():
    tanamans.append(
        Tanaman(
            jenis_tanaman = item['jenis_tanaman'],
            suhu = item['suhu'],
            suhu_interpolasi = item['suhu_interpolasi'],
            curah_hujan = item['curah_hujan'],
            curah_hujan_interpolasi = item['curah_hujan_interpolasi'],
            kelembapan = item['kelembapan'],
            kelembapan_interpolasi = item['kelembapan_interpolasi'],
            jenis_tanah = item['jenis_tanah'],
            tekstur_tanah = item['tekstur_tanah'],
            ph = item['ph'],
            ph_interpolasi = item['ph_interpolasi'],
            kemiringan = item['kemiringan'],
            kemiringan_interpolasi = item['kemiringan_interpolasi'],
            topografi = item['topografi'],
            topografi_interpolasi = item['topografi_interpolasi'],
        )
    )

In [33]:
try:
    db.bulk_save_objects(tanamans)
    db.commit()
    # db.refresh(tanamans)
except Exception as e:
    print(str(e))
    db.rollback()

## Ingest Data Lahan

In [4]:
import pandas as pd

df_lahan = pd.read_csv("../data/data_lahan_processed.csv")
df_lahan.head()

,no,desa,suhu,curah hujan,kelembapan,jenis tanah,tekstur tanah,ph,kemiringan,tinggi tempat
0,1,Kota Uneng,23.5,1023.5,62.5,mediteran,lempung berpasir sangat halus,6.75,2.0,37.5
1,2,Nangalimang,23.5,1023.5,62.5,mediteran,lempung berpasir sangat halus,6.75,2.0,50.0
2,3,Wuring,23.5,1023.5,62.5,mediteran,liat berpasir,5.10,35.5,250.0
3,4,Wolomarang,23.5,1023.5,62.5,mediteran,liat berpasir,5.10,35.5,250.0
4,5,Tilang,28.0,1725.0,80.0,regosol,liat,5.10,27.0,500.0


In [5]:
df_lahan.rename(columns={
    "no": "id",
    "curah hujan": "curah_hujan",
    "jenis tanah": "jenis_tanah",
    "tekstur tanah": "tekstur_tanah",
    "tinggi tempat": "topografi"
}, inplace=True)

In [6]:
df_lahan.head()

,id,desa,suhu,curah_hujan,kelembapan,jenis_tanah,tekstur_tanah,ph,kemiringan,topografi
0,1,Kota Uneng,23.5,1023.5,62.5,mediteran,lempung berpasir sangat halus,6.75,2.0,37.5
1,2,Nangalimang,23.5,1023.5,62.5,mediteran,lempung berpasir sangat halus,6.75,2.0,50.0
2,3,Wuring,23.5,1023.5,62.5,mediteran,liat berpasir,5.10,35.5,250.0
3,4,Wolomarang,23.5,1023.5,62.5,mediteran,liat berpasir,5.10,35.5,250.0
4,5,Tilang,28.0,1725.0,80.0,regosol,liat,5.10,27.0,500.0


In [7]:
df_lahan.to_sql("lahan", engine, index_label="id", index=False)

33